# Implement reconciliation

**We have to take into account the following:**

"In the construction of trade matrices, one should consider that the same trade flow can be reported twice in the FAOSTAT database, once by the exporting country and once by the importing country. When a trade flow is reported by only one of the two countries, the reported flow is used to construct the matrix (single record); this is the case for 40 % of records in the database. All other records are “double” (reported twice) and require a comparison between the declarations of the exporting and the importing countries, which are usually different, with a mean (absolute) relative difference, across all goods, countries, and years, of 61%. The choice of a value from two double records is called “reconciliation”, and the method adopted here is based on the identification of the most reliable reporting country among the two involved in each flow, and the use of the flow being reported by it. The reliability of countries is measured per commodity and per year with a data-based approach detailed below and adapted from Gehlhar (1996)." Source: https://doi.org/10.5194/essd-13-2025-2021 (they also provide guidance on country changes at the end of the supplementary material, see "Associations" at https://essd.copernicus.org/preprints/essd-2020-226/essd-2020-226-supplement.pdf)


The team behind the paper above computes a country reliability formula and selects which source of information to keep for duplicated entries (when both countries report the flow)


In [1]:
import pandas as pd
import numpy as np
import time

In [ ]:
start_time= time.time()
data = pd.read_csv('../Data/intermediate/Trade_filtered.csv', sep=',', encoding='utf-8',index_col=0)#.reset_index(drop=)
data.columns = data.columns.str.lower().str.replace(' ', '_')
data

,reporter_country_code,reporter_countries,partner_country_code,partner_countries,item,item_code,element,year,unit,value
0,2,Afghanistan,9,Argentina,"Cake, oilseeds nes",341,Import quantity,2017,t,463.11
1,2,Afghanistan,9,Argentina,"Cake, oilseeds nes",341,Import quantity,2019,t,1192.62
3,4,Algeria,2,Afghanistan,"Almonds, shelled",231,Import quantity,2005,t,3.00
5,2,Afghanistan,9,Argentina,"Cake, oilseeds nes",341,Import value,2017,1000 USD,85.00
6,4,Algeria,2,Afghanistan,"Almonds, shelled",231,Import value,2005,1000 USD,3.00
...,...,...,...,...,...,...,...,...,...,...
50428175,150,Netherlands (Kingdom of the),181,Zimbabwe,Wine,564,Export value,1988,1000 USD,1.00
50428176,150,Netherlands (Kingdom of the),181,Zimbabwe,Wine,564,Export value,1992,1000 USD,8.00
50428177,150,Netherlands (Kingdom of the),181,Zimbabwe,Wine,564,Export value,1999,1000 USD,2.00
50428178,150,Netherlands (Kingdom of the),181,Zimbabwe,Wine,564,Export value,2004,1000 USD,2.00


In [3]:
filter_t = data.item.unique()
filter_t

array(['Cake, oilseeds nes', 'Almonds, shelled', 'Beans, dry',
       'Anise, badian, coriander, cumin, caraway, fennel and juniper berries, raw',
       'Cider and other fermented beverages', 'Food preparations n.e.c.',
       'Wine', 'beef and veal preparations nes',
       'Poultry meat preparations', 'Almonds, in shell',
       'Meat of chickens, fresh or chilled', 'Coffee, green',
       'Crude organic material n.e.c.', 'Apricots, dried',
       'Refined sugar', 'Fruit prepared n.e.c.', 'Dates',
       'Cereal preparations', 'Sweet corn, frozen', 'Cabbages',
       'Avocados', 'Husked rice', 'Cigarettes', 'Carrots and turnips',
       'Pastry',
       'Chillies and peppers, green (Capsicum spp. and Pimenta spp.)',
       'Apple juice', 'Beer of barley, malted',
       'Cauliflowers and broccoli', 'Coconuts, desiccated', 'Apples',
       'Breakfast cereals',
       'Animal or vegetable fats and oils and their fractions, chemically modified, except those hydrogenated, inter-esterifi

In [4]:
# Define origin & destination
to_add = data.loc[:,['reporter_country_code','reporter_countries','partner_country_code','partner_countries']]
to_add.columns = ['origin_country_code','origin_country','destin_country_code','destin_country']
data=data.join(to_add)

# Invert origin and destination for imports
replace_bool = (data.element == 'Import quantity') | (data.element == 'Import value')
data.loc[replace_bool, ['origin_country', 'origin_country_code']] = data.loc[replace_bool, ['partner_countries', 'partner_country_code']].values
data.loc[replace_bool, ['destin_country', 'destin_country_code']] = data.loc[replace_bool, ['reporter_countries', 'reporter_country_code']].values

In [5]:
data.head(10)

,reporter_country_code,reporter_countries,partner_country_code,partner_countries,item,item_code,element,year,unit,value,origin_country_code,origin_country,destin_country_code,destin_country
0,2,Afghanistan,9,Argentina,"Cake, oilseeds nes",341,Import quantity,2017,t,463.11,9,Argentina,2,Afghanistan
1,2,Afghanistan,9,Argentina,"Cake, oilseeds nes",341,Import quantity,2019,t,1192.62,9,Argentina,2,Afghanistan
3,4,Algeria,2,Afghanistan,"Almonds, shelled",231,Import quantity,2005,t,3.00,2,Afghanistan,4,Algeria
5,2,Afghanistan,9,Argentina,"Cake, oilseeds nes",341,Import value,2017,1000 USD,85.00,9,Argentina,2,Afghanistan
6,4,Algeria,2,Afghanistan,"Almonds, shelled",231,Import value,2005,1000 USD,3.00,2,Afghanistan,4,Algeria
7,2,Afghanistan,9,Argentina,"Cake, oilseeds nes",341,Import value,2019,1000 USD,217.00,9,Argentina,2,Afghanistan
9,7,Angola,2,Afghanistan,"Beans, dry",176,Import quantity,2022,t,25.06,2,Afghanistan,7,Angola
10,4,Algeria,2,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",711,Import quantity,1996,t,171.00,2,Afghanistan,4,Algeria
11,7,Angola,2,Afghanistan,"Beans, dry",176,Import value,2022,1000 USD,39.00,2,Afghanistan,7,Angola
12,8,Antigua and Barbuda,2,Afghanistan,Cider and other fermented beverages,517,Import quantity,2012,t,0.00,2,Afghanistan,8,Antigua and Barbuda


### Reconciliation
Find repeated instances of code.

In [6]:
# Count double reports: takes a bit (3min)
data['double_reports']= 1*(data.groupby(["origin_country",'destin_country',"year","item","unit"], group_keys=False)['year'].transform('count')>1)
data['same_double_reports']= 1*(data.groupby(["origin_country",'destin_country',"year",'item','unit','value'], group_keys=False)["value"].transform('count')>1)

# Reliability estimation: (1.2 min)
reliability=data.groupby("reporter_countries", group_keys=False).apply(lambda x: sum(x.loc[:,'same_double_reports']) /sum(x.loc[:,'double_reports']),include_groups=False)
reliability=pd.DataFrame(reliability,columns=['reliability'])

data=pd.merge(data,reliability, left_on='reporter_countries',right_index=True)
print('Cum time (Reliability):', (time.time()- start_time)/60,'min')

Cum time (Reliability): 2.4576631784439087 min


In [7]:
# Remove where there is a double report and the country is not the preferred one 
# Find reliable country (can take up to 5min)

reliable_clean = data.sort_index().groupby(["origin_country",'destin_country',"year",'item','unit'], group_keys=False).tail(1)
reliable_clean=reliable_clean.drop(columns=['reporter_country_code', 'reporter_countries','partner_country_code','partner_countries','reliability','double_reports','same_double_reports'])


# Save data
reliable_clean.to_csv('../Data/intermediate/Trade_reconciled.csv',encoding='utf-8',index=False) # Checked data clean same dimensions than R code need to check values
print('Cum time (after save):', (time.time()- start_time)/60,'min')
reliable_clean.head(10)  

Cum time (after save): 4.654310667514801 min


,item,item_code,element,year,unit,value,origin_country_code,origin_country,destin_country_code,destin_country
0,"Cake, oilseeds nes",341,Import quantity,2017,t,463.11,9,Argentina,2,Afghanistan
1,"Cake, oilseeds nes",341,Import quantity,2019,t,1192.62,9,Argentina,2,Afghanistan
3,"Almonds, shelled",231,Import quantity,2005,t,3.00,2,Afghanistan,4,Algeria
5,"Cake, oilseeds nes",341,Import value,2017,1000 USD,85.00,9,Argentina,2,Afghanistan
6,"Almonds, shelled",231,Import value,2005,1000 USD,3.00,2,Afghanistan,4,Algeria
7,"Cake, oilseeds nes",341,Import value,2019,1000 USD,217.00,9,Argentina,2,Afghanistan
9,"Beans, dry",176,Import quantity,2022,t,25.06,2,Afghanistan,7,Angola
10,"Anise, badian, coriander, cumin, caraway, fenn...",711,Import quantity,1996,t,171.00,2,Afghanistan,4,Algeria
11,"Beans, dry",176,Import value,2022,1000 USD,39.00,2,Afghanistan,7,Angola
12,Cider and other fermented beverages,517,Import quantity,2012,t,0.00,2,Afghanistan,8,Antigua and Barbuda
